In [139]:
import json
import pandas as pd
import plotly.express as px
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import unicodedata

In [140]:

# Función para normalizar nombres de canciones (quita espacios extra y pasa a minúsculas)
def normalize(song):
    song = song.strip().lower()
    song = ''.join(c for c in unicodedata.normalize('NFKD', song) if not unicodedata.combining(c) or c == 'ñ')
    return song

# Cargar los archivos JSON
with open(r'C:\Users\Yo\Desktop\setlist\SetlistFM-Setlist-Evolution\album_data\Los Piojos.json', 'r', encoding='utf-8') as f:
    albums_data = json.load(f)

with open('Setlists_Piojos.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)

# Crear diccionarios para mapear cada canción a su álbum y a su color
song_to_album = {}
song_to_color = {}
for album in albums_data.get("albums", []):
    album_title = album.get("title", "Ineditos")
    color = album.get("color", "gray")
    for track in album.get("tracks", []):
        norm_track = normalize(track)
        # Si la canción aparece en varios álbumes, se asigna el primero encontrado
        if norm_track not in song_to_album:
            song_to_album[norm_track] = album_title
            song_to_color[norm_track] = color

# Crear un diccionario que mapea cada álbum a su color en formato HEX
album_to_color = {}
for album in albums_data.get("albums", []):
    album_to_color[album.get("title", "Ineditos")] = album.get("color", "#39df00")


# Ordenar los setlists por fecha (usando year, month, day)
setlists_data.sort(key=lambda s: date(s["year"], s["month"], s["day"]))

# Procesar cada setlist y contar acumulativamente
records = []
cumulative_counts = {}

for setlist in setlists_data:
    # Crear fecha en formato ISO
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    
    # Procesar la lista de canciones, separando por "/" si es necesario
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    
    # Actualizar los conteos acumulados
    for song in canciones:
        norm_song = normalize(song)
        cumulative_counts[norm_song] = cumulative_counts.get(norm_song, 0) + 1
    # Registrar el estado acumulado hasta la fecha actual
    for song, count in cumulative_counts.items():
        album = song_to_album.get(song, "Ineditos")
        color = song_to_color.get(song, "#000000")
        records.append({
            "fecha": current_date,
            "canción": song.title(),  # Capitalizamos para visualización
            "álbum": album,
            "color": color,
            "veces_tocada": count,

        })

# Crear DataFrame y agrupar para quedarnos con el último conteo de cada fecha
df = pd.DataFrame(records)


# Diccionario de colores corregido
colores_album = {
    "Ay Ay Ay": "#c41616",
    "3er Arco": "#ead936",
    "Chac Tu Chac": "#1e2830",
    "Azul": "#0047a5",
    "Verde Paisaje del Infierno": "#648e5c",
    "Maquina de Sangre": "#d57f5c",
    "Civilizacion": "#d7c391",
    "Ineditos": "#b3b3b3"
}

# Reemplazar los valores incorrectos en la columna 'color'
df["color"] = df["álbum"].map(colores_album)
df["álbum"] = df["álbum"].astype("category")

# Crear un DataFrame con todas las canciones en la fecha inicial con 0 reproducciones
fecha_inicio = df["fecha"].min()
df_inicio = df.copy()
df_inicio["fecha"] = fecha_inicio  # Asigna la fecha inicial
df_inicio["veces_tocada"] = 0      # Todas las canciones con 0

# Concatenamos con el DataFrame original
df_final = pd.concat([df_inicio, df], ignore_index=True)


In [141]:
df_final

,fecha,canción,álbum,color,veces_tocada
0,1991-09-21,Blues Del Gato Sarnoso,Ineditos,#b3b3b3,0
1,1991-09-21,Los Mocosos,Chac Tu Chac,#1e2830,0
2,1991-09-21,Blues Del Traje Gris,Chac Tu Chac,#1e2830,0
3,1991-09-21,Chac Tu Chac,Chac Tu Chac,#1e2830,0
4,1991-09-21,Babilonia,Ay Ay Ay,#c41616,0
...,...,...,...,...,...
55423,2025-05-25,Superstition,Ineditos,#b3b3b3,3
55424,2025-05-25,Sabado Noche,Ineditos,#b3b3b3,1
55425,2025-05-25,¡Me Matan Limon!,Ineditos,#b3b3b3,1
55426,2025-05-25,Juana Azurduy,Ineditos,#b3b3b3,3


In [142]:
df_canciones = pd.DataFrame(albums_data["albums"])
df_canciones = df_canciones.explode('tracks').reset_index(drop=True)

df_canciones

,title,color,tracks
0,Chac Tu Chac,#1e2830,Llévatelo
1,Chac Tu Chac,#1e2830,Chac Tu Chac
2,Chac Tu Chac,#1e2830,Tan Solo
3,Chac Tu Chac,#1e2830,Cancheros
4,Chac Tu Chac,#1e2830,Los Mocosos
...,...,...,...
155,Ineditos,#d57f5c,Silver Moon
156,Ineditos,#d57f5c,Superstition
157,Ineditos,#d57f5c,Sábado Noche
158,Ineditos,#d57f5c,Juana Azurduy


In [143]:
# Para cada canción, obtener la fila con el máximo 'veces_tocada' (último conteo acumulado)
idx = df_final.groupby("canción")["veces_tocada"].idxmax()
df_final_max = df_final.loc[idx].reset_index(drop=True)

df_final_max['canción'] = df_final_max['canción'].apply(lambda x: x.title())  # Capitalizar nombres de canciones
df_final_max['canción'] = df_final_max['canción'].apply(normalize)  # Normalizar nombres de canciones
df_final_max



,fecha,canción,álbum,color,veces_tocada
0,2000-07-09,a la huella,Ineditos,#b3b3b3,1
1,2024-12-21,a veces,Chac Tu Chac,#1e2830,23
2,2008-12-11,a ver cuando,Azul,#0047a5,9
3,2025-05-03,agua,Azul,#0047a5,57
4,2006-08-24,al atardecer,3er Arco,#ead936,11
...,...,...,...,...,...
156,2025-05-03,y quemas,Azul,#0047a5,31
157,2025-05-03,yira yira,Chac Tu Chac,#1e2830,58
158,2025-04-24,you gotta move,Ineditos,#b3b3b3,3
159,2025-01-25,zapatos de gamuza azul,Ineditos,#b3b3b3,35


In [144]:
# Obtener la primera fecha donde se tocó cada canción (veces_tocada > 0)
primeras_fechas = (
    df_final[df_final['veces_tocada'] > 0]
    .groupby('canción')['fecha']
    .min()
    .reset_index()
    .rename(columns={'fecha': 'primer_fecha'})
)

primeras_fechas


,canción,primer_fecha
0,A La Huella,2000-07-09
1,A Veces,1992-11-14
2,A Ver Cuando,1998-05-16
3,Agua,1997-07-18
4,Al Atardecer,1996-09-28
...,...,...
156,Y Quemas,1998-05-17
157,Yira Yira,1991-09-21
158,You Gotta Move,1991-09-21
159,Zapatos De Gamuza Azul,1992-11-14


In [145]:

df_final_max[df_final_max['canción'] == 'llevatelo']

,fecha,canción,álbum,color,veces_tocada
86,2025-05-25,llevatelo,Chac Tu Chac,#1e2830,45


In [146]:
df_canciones = df_canciones.rename(columns={"tracks": "canción"})
df_canciones = df_canciones.rename(columns={"title": "álbum"})
df_canciones = df_canciones.rename(columns={"color": "color_álbum"})
df_canciones['canción'] = df_canciones['canción'].apply(lambda x: x.title())  # Capitalizar nombres de canciones
df_canciones['canción'] = df_canciones['canción'].apply(normalize)  # Normalizar nombres de canciones

df_canciones = df_canciones.merge(df_final_max, on=["canción", "álbum"], how="left")


In [147]:
df_canciones['canción'] = df_canciones['canción'].str.title()
df_canciones

,álbum,color_álbum,canción,fecha,color,veces_tocada
0,Chac Tu Chac,#1e2830,Llevatelo,2025-05-25,#1e2830,45.0
1,Chac Tu Chac,#1e2830,Chac Tu Chac,2025-04-13,#1e2830,67.0
2,Chac Tu Chac,#1e2830,Tan Solo,2025-05-25,#1e2830,126.0
3,Chac Tu Chac,#1e2830,Cancheros,2025-04-26,#1e2830,27.0
4,Chac Tu Chac,#1e2830,Los Mocosos,2025-05-03,#1e2830,46.0
...,...,...,...,...,...,...
155,Ineditos,#d57f5c,Silver Moon,2008-11-08,#b3b3b3,1.0
156,Ineditos,#d57f5c,Superstition,2009-04-04,#b3b3b3,3.0
157,Ineditos,#d57f5c,Sabado Noche,2025-04-13,#b3b3b3,1.0
158,Ineditos,#d57f5c,Juana Azurduy,2025-05-25,#b3b3b3,3.0


In [148]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Configura tus credenciales de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id='f08cac877ee344a39c4e3b8044a467bc',
    client_secret='b70dee3e801f4a71b0282f39383cc4f1'
))

'''def get_spotify_plays(song, artist="Los Piojos"):
    query = f"track:{song} artist:{artist}"
    results = sp.search(q=query, type='track', limit=1)
    items = results['tracks']['items']
    if items:
        return items[0]['popularity']  # No hay plays exactos, pero sí popularidad (0-100)
    return None'''

def get_spotify_plays(song, artist="Los Piojos"):
    """
    Busca la popularidad de una canción en Spotify.
    Si no encuentra con el artista principal, prueba solo por nombre de canción.
    Si tampoco encuentra, intenta buscar sin artista (puede ser cover o versión).
    Devuelve None si no encuentra ningún resultado.
    """
    # 1. Busca con artista principal
    query = f"track:{song} artist:{artist}"
    results = sp.search(q=query, type='track', limit=1)
    items = results['tracks']['items']
    if items:
        return items[0]['popularity']
    # 2. Busca solo por nombre de canción (puede ser cover, versión, etc.)
    results = sp.search(q=f"track:{song}", type='track', limit=1)
    items = results['tracks']['items']
    if items:
        return items[0]['popularity']
    # 3. Busca por nombre exacto (sin filtro de tipo)
    results = sp.search(q=song, type='track', limit=1)
    items = results['tracks']['items']
    if items:
        return items[0]['popularity']
    # No encontrado
    return None

df_canciones['spotify_popularity'] = df_canciones['canción'].apply(get_spotify_plays)


In [149]:
df_canciones['spotify_popularity'].median()

45.0

In [150]:
import yt_dlp
import pandas as pd

def get_youtube_views(query, n=5):
    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'extract_flat': 'in_playlist',
        'default_search': f'ytsearch{n}',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(query, download=False)
            if 'entries' in info and info['entries']:
                # Suma las vistas de los primeros n videos
                total_views = sum(entry.get('view_count', 0) for entry in info['entries'][:n])
                return total_views
        except Exception:
            return None
    return None

# Ahora buscará los primeros 5 videos y sumará sus vistas
df_canciones['reprod_youtube'] = df_canciones.apply(
    lambda row: get_youtube_views(f"{row['canción']} Los Piojos", n=3), axis=1
)

In [151]:
df_canciones

,álbum,color_álbum,canción,fecha,color,veces_tocada,spotify_popularity,reprod_youtube
0,Chac Tu Chac,#1e2830,Llevatelo,2025-05-25,#1e2830,45.0,46,989350
1,Chac Tu Chac,#1e2830,Chac Tu Chac,2025-04-13,#1e2830,67.0,50,1162120
2,Chac Tu Chac,#1e2830,Tan Solo,2025-05-25,#1e2830,126.0,69,15922263
3,Chac Tu Chac,#1e2830,Cancheros,2025-04-26,#1e2830,27.0,45,584696
4,Chac Tu Chac,#1e2830,Los Mocosos,2025-05-03,#1e2830,46.0,49,1962612
...,...,...,...,...,...,...,...,...
155,Ineditos,#d57f5c,Silver Moon,2008-11-08,#b3b3b3,1.0,37,47018953
156,Ineditos,#d57f5c,Superstition,2009-04-04,#b3b3b3,3.0,77,9150
157,Ineditos,#d57f5c,Sabado Noche,2025-04-13,#b3b3b3,1.0,41,2435142
158,Ineditos,#d57f5c,Juana Azurduy,2025-05-25,#b3b3b3,3.0,29,1916185


In [152]:
data = '''
Song Title	Streams	Daily
Tan Solo 102,268,899	79,506
Bicho de Ciudad 64,948,157	69,354
El Farolito 64,352,521	66,936
Como Alí 47,591,515	46,178
Ruleta 45,015,653	41,330
Ando Ganas (Llora Llora) 42,467,530	28,109
Verano del 92 37,892,518	28,255
Todo Pasa 31,649,085	21,513
Civilización 28,167,736	21,715
Vine Hasta Aquí 21,854,601	31,423
Canción de Cuna 20,178,108	13,662
Pacífico 19,232,459	19,372
Desde Lejos No Se Ve 17,583,199	23,000
Muy Despacito 16,249,568	11,006
Amor de Perros 16,040,522	7,199
Pistolas 14,308,552	19,240
Muévelo 12,512,312	13,040
Agua 11,295,121	10,494
El Balneario de los Doctores Crotos 10,686,329	10,927
Yira Yira 10,461,357	9,075
Al Atardecer 8,899,107	3,939
Taxi Boy 8,733,004	12,167
Te Diría 8,611,297	13,011
Marado 8,308,121	8,081
Y Quemás 6,045,849	4,522
Difícil 5,899,333	11,231
Esquina Libertad 5,862,629	5,952
Babilonia 5,047,786	8,549
Luz de Marfil 4,887,935	9,425
Cruel 4,844,211	6,364
Buenos Tiempos 4,484,190	8,422
Arco 4,443,314	6,219
Genius 4,166,304	6,029
Dientes de Cordero 4,154,862	4,356
Sudestada 4,144,062	9,250
Los Mocosos 4,075,659	4,807
Ay Ay Ay 3,953,398	4,026
Fantasma 3,908,783	5,963
Chac Tu Chac 3,809,631	5,654
Shup Shup 3,469,845	3,951
Pensar en Nada 3,365,805	1,285
Tan Solo 3,307,141	1,790
La Luna y la Cabra 3,241,404	5,514
Fijate 3,205,038	2,487
Basta de Penas 3,026,720	3,655
Morella 2,952,514	6,699
Labios de Seda 2,906,677	3,684
A Veces 2,797,426	5,654
Gris 2,586,045	1,531
Llevátelo 2,525,463	3,754
Buenos Días Palomar 2,524,614	2,426
Manjar 2,444,019	2,664
Angelito 2,396,411	3,589
Intro Marado 2,375,751	1,369
A Ver Cuando 2,208,234	2,834
Que Decís 2,202,626	1,506
Hoy es Hoy 2,044,482	3,899
Fumigator 2,023,695	3,445
Cancheros 1,990,564	3,712
Don't Say Tomorrow 1,951,824	3,153
Entrando en Tu Ciudad 1,950,431	2,575
María y José 1,939,651	4,325
Uoh Pa Pa Pa 1,889,845	1,650
Quemado 1,868,693	1,273
Te Diría - En Vivo 1,818,573	967
Media Caña 1,743,210	2,162
Murguita 1,685,532	2,544
Agua 1,621,718	689
Manise 1,621,104	2,187
Es Sentir 1,595,436	1,934
Chac Tu Chac 1,580,889	838
Fantasma - En Vivo 1,520,388	1,261
Finale 1,506,119	1,390
Siempre Bajando 1,500,029	1,880
Labios de Seda 1,488,049	3,299
Olvidate 1,452,986	2,009
Pollo Viejo 1,416,027	1,515
Go Negro Go 1,378,693	2,046
Guadalupe 1,330,033	1,471
Babilonia 1,329,910	1,927
Labios de Seda 1,318,750	584
Vals Inicial 1,315,486	4,364
Motumbo 1,299,141	3,552
El Rey del Blues 1,292,078	1,515
Sucio Can 1,261,600	1,141
Ximenita 1,240,467	1,210
Blues del Traje Gris 1,232,248	1,927
Te Diría 1,226,387	2,293
Cruces y Flores 1,221,919	1,406
Desde Lejos No Se Ve - En Vivo 1,165,587	701
Unbekannt 1,154,163	5,938
Pistolas - En Vivo 1,128,065	1,779
Reggae Rojo y Negro 1,122,759	2,034
Babilonia - En Vivo 1,121,689	792
Maradó - En Vivo 1,098,509	522
Un Buen Día 1,089,920	1,228
María y José 1,079,700	847
Manise 1,061,632	2,649
Globalización 1,056,355	1,494
Pacífico 1,048,527	2,674
Salitral 1,040,291	1,250
Arco II 1,039,861	1,508
Luz de Marfil 1,032,783	1,845
Ando Ganas (Llora Llora) 1,029,864	802
No Parés 983,051	1,412
Ando Ganas 966,945	1,847
Pega Pega 962,690	1,061
San Jauretche 952,499	1,676
Morella 908,638	557
Olvidate (Ya Ves) 889,774	944
Fijate 873,556	1,248
Angelito - En Vivo 858,149	423
Todo Pasa 846,078	708
Esquina Libertad 844,979	1,361
Manjar 828,211	2,457
Guadalupe - En Vivo 810,373	362
Todo Pasa 767,852	1,517
El Farolito (La Rubia Tarada) 754,193	993
Desde Lejos No Se Ve 746,310	883
Merecido 741,420	2,864
Langostas 741,358	1,854
Genius (El Mendigo de Dock Sud) 731,615	1,439
Muy Despacito 728,718	757
Ruleta - En Vivo 717,251	850
Sudestada 708,596	1,920
Difícil 706,985	1,903
Solo y en Paz 692,052	881
Al Desierto 691,832	931
Dientes de Cordero - En Vivo 684,180	455
Mi Babe 680,077	1,131
Luz de Marfil - En Vivo 671,681	1,223
Yira Yira 636,373	867
Y Que Más 607,535	764
Cruel 580,953	798
Desde Lejos No Se Ve 569,544	1,244
Cruel 564,243	1,364
Verano del 92 549,630	1,245
Ay Ay Ay 536,990	887
El Balneario de los Doctores Crotos 532,155	1,411
Llévatelo 531,318	1,157
Arco 524,446	716
El Balneario de los Doctores Crotos 523,482	628
Taxi Boy - En Vivo 518,633	568
Ruleta 514,351	1,283
Canción de Cuna 508,180	1,163
Todo Pasa - En Vivo 477,048	461
Amor de Perros - En Vivo 468,999	496
Buenos Días Palomar 456,421	932
Cruel - En Vivo 456,204	523
Taxi Boy 453,248	488
Morella - En Vivo 444,922	666
Los Mocosos 444,102	953
Ay Ay Ay - En Vivo 442,818	499
A Veces 440,905	502
Cancheros 435,339	676
Extraña Soledad 431,136	647
Muévelo 430,248	1,157
El Farolito - En Vivo 428,465	514
Maradó 424,765	388
Babilonia 418,808	514
Angelito 414,265	494
Como Alí - En Vivo 410,519	471
Motumbo - En Vivo 408,412	673
Canción de Cuna - En Vivo 403,387	307
Chac Tu Chac - En Vivo 398,555	391
Media Caña - En Vivo 393,209	542
Around & Around / Zapatos de Gamuza Azul 385,085	480
El Viejo 374,401	753
Tan Solo - En Vivo 368,692	491
Ximenita 365,615	471
Es Sólo Rock & Roll (It´s Only Rock & Roll) 359,986	876
Intro Maradó 348,670	290
Quemado - En Vivo 337,527	339
Fíjate 337,312	389
Fijate - En Vivo 313,879	335
El Viejo - En Vivo 308,636	290
Finale 299,962	603
El Rey del Blues (B. B. King) 296,340	329
Llevatelo - En Vivo 268,965	228
Langostas - En Vivo 259,955	563
No Pares - En Vivo 256,212	259
Little Red Rooster (Zapada) 191,721	219
'''
import re

rows = []
for line in data.strip().split('\n'):
    # Busca el nombre de la canción y el primer número (Streams)
    match = re.match(r"(.+?)\s+([\d,]+)\s+[\d,]+$", line.strip())
    if match:
        title = match.group(1)
        streams = match.group(2).replace(',', '')
        try:
            streams = int(streams)
        except:
            continue
        rows.append([title, streams])

df_streams = pd.DataFrame(rows, columns=['Song Title', 'Streams'])
df_streams[df_streams['Streams'] > 30000000]

,Song Title,Streams
0,Tan Solo,102268899
1,Bicho de Ciudad,64948157
2,El Farolito,64352521
3,Como Alí,47591515
4,Ruleta,45015653
5,Ando Ganas (Llora Llora),42467530
6,Verano del 92,37892518
7,Todo Pasa,31649085


In [153]:

import unicodedata

def normalize(song):
    song = song.strip().lower()
    song = song.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u").replace("ñ", "n")
    song = song.replace("-", " ").replace("'", "").replace("’", "")
    song = song.replace("(", "").replace(")", "").replace("[", "").replace("]", "")
    song = song.replace(",", " ").replace(";", " ").replace(":", " ")
    song = ''.join(c for c in unicodedata.normalize('NFKD', song) if not unicodedata.combining(c))
    song = song.replace(" en vivo", "").replace("(la rubia tarada)", "").replace("(b. b. king)", "").replace("(zapada)", "")
    song = song.replace("(el mendigo de dock sud)", "")
    song = song.replace("  ", " ").strip()
    return song

# Canciones de tu df (normalizadas)
canciones_df = [normalize(c) for c in df_canciones['canción'].unique()]


In [154]:
# Normaliza y agrupa las reproducciones de df_streams
from collections import defaultdict

repro_sum = defaultdict(int)
for _, row in df_streams.iterrows():
    norm = normalize(row['Song Title'])
    repro_sum[norm] += row['Streams']

In [155]:
# Crea la lista final solo con las canciones de tu df
data_filtrada = []
for c in canciones_df:
    if c in repro_sum:
        data_filtrada.append([c.title(), repro_sum[c]])

In [156]:
df = pd.DataFrame(data_filtrada, columns=["canción", "reproducciones_spotify"])


df

,canción,reproducciones_spotify
0,Llevatelo,3325746
1,Chac Tu Chac,5789075
2,Tan Solo,105944732
3,Cancheros,2425903
4,Los Mocosos,4519761
...,...,...
92,Buenos Dias Palomar,2981035
93,Olvidate Ya Ves,889774
94,Extrana Soledad,431136
95,Pensar En Nada,3365805


In [157]:
# Crea columna auxiliar normalizada en ambos DataFrames
df_canciones['cancion_norm'] = df_canciones['canción'].apply(normalize)

df['cancion_norm'] = df['canción'].apply(normalize)

# Merge usando la columna normalizada
df_canciones = df_canciones.merge(df[['cancion_norm', 'reproducciones_spotify']], on='cancion_norm', how='left')

# Elimina la columna auxiliar si no la necesitas
df_canciones = df_canciones.drop(columns=['cancion_norm'])


In [158]:
# Aplicar a cada álbum una nueva columna con el número de ventas del álbum
df_canciones['ventas'] = df_canciones['álbum'].map({
    "Ay Ay Ay": 60000,
    "3er Arco": 360000,
    "Chac Tu Chac": 60000,
    "Azul": 120000,
    "Verde Paisaje del Infierno": 120000,
    "Maquina de Sangre": 80000,
    "Civilizacion": 80000,
    "Ineditos": 80000
})

In [159]:
df_canciones

,álbum,color_álbum,canción,fecha,color,veces_tocada,spotify_popularity,reprod_youtube,reproducciones_spotify,ventas
0,Chac Tu Chac,#1e2830,Llevatelo,2025-05-25,#1e2830,45.0,46,989350,3325746.0,60000
1,Chac Tu Chac,#1e2830,Chac Tu Chac,2025-04-13,#1e2830,67.0,50,1162120,5789075.0,60000
2,Chac Tu Chac,#1e2830,Tan Solo,2025-05-25,#1e2830,126.0,69,15922263,105944732.0,60000
3,Chac Tu Chac,#1e2830,Cancheros,2025-04-26,#1e2830,27.0,45,584696,2425903.0,60000
4,Chac Tu Chac,#1e2830,Los Mocosos,2025-05-03,#1e2830,46.0,49,1962612,4519761.0,60000
...,...,...,...,...,...,...,...,...,...,...
155,Ineditos,#d57f5c,Silver Moon,2008-11-08,#b3b3b3,1.0,37,47018953,NaN,80000
156,Ineditos,#d57f5c,Superstition,2009-04-04,#b3b3b3,3.0,77,9150,NaN,80000
157,Ineditos,#d57f5c,Sabado Noche,2025-04-13,#b3b3b3,1.0,41,2435142,NaN,80000
158,Ineditos,#d57f5c,Juana Azurduy,2025-05-25,#b3b3b3,3.0,29,1916185,NaN,80000


In [160]:
# Supón que tienes el año de lanzamiento en una columna 'año_lanzamiento'
import datetime
# Si tienes df_final con todas las canciones y fechas
# Ahora puedes recalcular años_vigencia normalmente
# Aplicar a cada álbum una nueva columna con el número de ventas del álbum
df_canciones['año_lanzamiento'] = df_canciones['álbum'].map(
    {
    "Ay Ay Ay": 1994,
    "3er Arco": 1996,
    "Chac Tu Chac": 1992,
    "Azul": 1998,
    "Verde Paisaje del Infierno": 2000,
    "Maquina de Sangre": 2003,
    "Civilizacion": 2007
}
)

ineditos = df_canciones[df_canciones['álbum'] == 'Ineditos']['canción'].unique()
for cancion in ineditos:
    fechas = primeras_fechas[primeras_fechas['canción'] == cancion]['primer_fecha']
    if not fechas.empty:
        año_primera = int(fechas.min()[:4])
        df_canciones.loc[(df_canciones['canción'] == cancion) & (df_canciones['álbum'] == 'Ineditos'), 'año_lanzamiento'] = año_primera
    else:
        df_canciones.loc[(df_canciones['canción'] == cancion) & (df_canciones['álbum'] == 'Ineditos'), 'año_lanzamiento'] = 1991  # fallback




año_actual = datetime.datetime.now().year

# Si quieres restar 15 años solo a canciones lanzadas antes de 2024, usa np.where
import numpy as np
df_canciones['años_vigencia'] = np.where(
    df_canciones['año_lanzamiento'] < 2024,
    año_actual - df_canciones['año_lanzamiento'] - 15,
    abs(año_actual - df_canciones['año_lanzamiento'] - 1)
)
# años de separación

# Nueva métrica: veces tocada por año de vigencia
df_canciones['veces_tocada_por_año'] = df_canciones['veces_tocada'] / df_canciones['años_vigencia']

df_canciones

,álbum,color_álbum,canción,fecha,color,veces_tocada,spotify_popularity,reprod_youtube,reproducciones_spotify,ventas,año_lanzamiento,años_vigencia,veces_tocada_por_año
0,Chac Tu Chac,#1e2830,Llevatelo,2025-05-25,#1e2830,45.0,46,989350,3325746.0,60000,1992.0,18.0,2.500000
1,Chac Tu Chac,#1e2830,Chac Tu Chac,2025-04-13,#1e2830,67.0,50,1162120,5789075.0,60000,1992.0,18.0,3.722222
2,Chac Tu Chac,#1e2830,Tan Solo,2025-05-25,#1e2830,126.0,69,15922263,105944732.0,60000,1992.0,18.0,7.000000
3,Chac Tu Chac,#1e2830,Cancheros,2025-04-26,#1e2830,27.0,45,584696,2425903.0,60000,1992.0,18.0,1.500000
4,Chac Tu Chac,#1e2830,Los Mocosos,2025-05-03,#1e2830,46.0,49,1962612,4519761.0,60000,1992.0,18.0,2.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Ineditos,#d57f5c,Silver Moon,2008-11-08,#b3b3b3,1.0,37,47018953,NaN,80000,2008.0,2.0,0.500000
156,Ineditos,#d57f5c,Superstition,2009-04-04,#b3b3b3,3.0,77,9150,NaN,80000,2008.0,2.0,1.500000
157,Ineditos,#d57f5c,Sabado Noche,2025-04-13,#b3b3b3,1.0,41,2435142,NaN,80000,2025.0,1.0,1.000000
158,Ineditos,#d57f5c,Juana Azurduy,2025-05-25,#b3b3b3,3.0,29,1916185,NaN,80000,2025.0,1.0,3.000000


In [161]:
df_canciones[["veces_tocada", "spotify_popularity", "reprod_youtube", "reproducciones_spotify", "ventas", 'veces_tocada_por_año']].corr()


,veces_tocada,spotify_popularity,reprod_youtube,reproducciones_spotify,ventas,veces_tocada_por_año
veces_tocada,1.000000,0.263441,-0.029357,0.505838,0.336240,0.707822
spotify_popularity,0.263441,1.000000,0.185920,0.789214,0.080713,0.241483
reprod_youtube,-0.029357,0.185920,1.000000,0.596966,-0.031935,-0.031935
reproducciones_spotify,0.505838,0.789214,0.596966,1.000000,0.114252,0.404401
ventas,0.336240,0.080713,-0.031935,0.114252,1.000000,0.143612
veces_tocada_por_año,0.707822,0.241483,-0.031935,0.404401,0.143612,1.000000


In [162]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# Selecciona las columnas a usar en el índice
cols = [
    "veces_tocada_por_año",
    "spotify_popularity",
    "reprod_youtube",
    "reproducciones_spotify",
    "ventas"
]


# Z-score (media 0, std 1)
scaler_z = StandardScaler()
df_popularidad = df_canciones.copy()
df_popularidad[[f"{c}_z" for c in cols]] = scaler_z.fit_transform(df_canciones[cols].fillna(0))

# --- 4. Calculamos los Pesos---

# Usamos PCA para calcular los pesos de cada variable (en la escala z-score)
pca = PCA(n_components=len(cols))
X_z = df_popularidad[[f"{c}_z" for c in cols]].fillna(0)
pca.fit(X_z)

# Tomamos la varianza explicada por cada componente y el peso absoluto de cada variable en el primer componente
component_1 = np.abs(pca.components_[0])
pesos_pca = component_1 / component_1.sum()

# Asociamos cada peso a su columna z-score
pesos_dict = {f"{col}_z": peso for col, peso in zip(cols, pesos_pca)}
# Puedes definir dos diccionarios de pesos diferentes
pesos_dict_album = {f"{col}_z": peso for col, peso in zip(cols, pesos_pca)}
pesos_dict_ineditos = {
    'veces_tocada_por_año_z': 0.4,
    'spotify_popularity_z': 0.2,
    'reprod_youtube_z': 0.2,
    'reproducciones_spotify_z': 0.1,
    'ventas_z': 0.1
}

In [163]:
# --- 5. Mostramos ---
# Calcula el índice de popularidad usando pesos diferentes según el álbum
def calcular_indice_popularidad(row):
    if row['álbum'] == 'Ineditos':
        pesos = pesos_dict_ineditos
    else:
        pesos = pesos_dict
    return sum(row[col] * peso for col, peso in pesos.items())

df_popularidad['indice_popularidad'] = df_popularidad.apply(calcular_indice_popularidad, axis=1)
df_popularidad[['álbum', 'canción', 'indice_popularidad'] + [f"{c}_z" for c in cols]]

# Normaliza el índice de popularidad entre 0 y 1
min_val = df_popularidad['indice_popularidad'].min()
max_val = df_popularidad['indice_popularidad'].max()
df_popularidad['indice_popularidad_norm'] = (df_popularidad['indice_popularidad'] - min_val) / (max_val - min_val)



df_popularidad.sort_values(by='indice_popularidad_norm', ascending=False).head()

,álbum,color_álbum,canción,fecha,color,veces_tocada,spotify_popularity,reprod_youtube,reproducciones_spotify,ventas,año_lanzamiento,años_vigencia,veces_tocada_por_año,veces_tocada_por_año_z,spotify_popularity_z,reprod_youtube_z,reproducciones_spotify_z,ventas_z,indice_popularidad,indice_popularidad_norm
2,Chac Tu Chac,#1e2830,Tan Solo,2025-05-25,#1e2830,126.0,69,15922263,105944732.0,60000,1992.0,18.0,7.000000,1.180062,1.622176,0.551958,7.491324,-0.610634,2.876289,1.000000
83,Civilizacion,#d7c391,Bicho De Ciudad,2025-05-25,#d7c391,52.0,69,4262418,64948157.0,80000,2007.0,3.0,17.333333,4.046997,1.622176,-0.104704,4.431157,-0.352165,2.710580,0.957570
26,3er Arco,#ead936,El Farolito,2025-05-25,#ead936,120.0,68,20467350,64780986.0,360000,1996.0,14.0,8.571429,1.616048,1.547010,0.807929,4.418678,3.266410,2.600115,0.929285
60,Verde Paisaje del Infierno,#648e5c,Ruleta,2025-05-25,#648e5c,88.0,66,7532476,46247255.0,120000,2000.0,10.0,8.800000,1.679464,1.396678,0.079460,3.035238,0.164774,1.701312,0.699145
81,Civilizacion,#d7c391,Pacifico,2025-05-24,#d7c391,54.0,60,11795870,20280986.0,80000,2007.0,3.0,18.000000,4.231961,0.945682,0.319567,1.097000,-0.352165,1.637807,0.682885


In [164]:
df_popularidad.to_excel('canciones_popularidad.xlsx', index=False, header=True)

----------------------

In [165]:
# Función para normalizar nombres de canciones (quita espacios extra y pasa a minúsculas)
def normalize(song):
    song = song.strip().lower()
    song = ''.join(c for c in unicodedata.normalize('NFKD', song) if not unicodedata.combining(c) or c == 'ñ')
    return song



with open('setlists_vuelta.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)




# Ordenar los setlists por fecha (usando year, month, day)
setlists_data.sort(key=lambda s: date(s["year"], s["month"], s["day"]))

# Procesar cada setlist y contar acumulativamente
records = []
cumulative_counts = {}

for setlist in setlists_data:
    # Crear fecha en formato ISO
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    
    # Procesar la lista de canciones, separando por "/" si es necesario
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    
    # Actualizar los conteos acumulados
    for song in canciones:
        norm_song = normalize(song)
        cumulative_counts[norm_song] = cumulative_counts.get(norm_song, 0) + 1
    # Registrar el estado acumulado hasta la fecha actual
    for song, count in cumulative_counts.items():
        album = song_to_album.get(song, "Ineditos")
        color = song_to_color.get(song, "#000000")
        records.append({
            "fecha": current_date,
            "canción": song.title(),  # Capitalizamos para visualización
            "álbum": album,
            "color": color,
            "veces_tocada": count,

        })

# Crear DataFrame y agrupar para quedarnos con el último conteo de cada fecha
df_vuelta_total = pd.DataFrame(records)


df_vuelta_total[df_vuelta_total['fecha'] == '2025-05-25'].head()

,fecha,canción,álbum,color,veces_tocada
795,2025-05-25,Te Diria,Ay Ay Ay,#c41616,8
796,2025-05-25,Desde Lejos No Se Ve,Azul,#0047a5,8
797,2025-05-25,Babilonia,Ay Ay Ay,#c41616,5
798,2025-05-25,Ay Ay Ay,Ay Ay Ay,#c41616,14
799,2025-05-25,Todo Pasa,3er Arco,#ead936,3


In [166]:
import json
from datetime import date

# Asume que song_to_album y song_to_color ya están definidos

with open('setlists_vuelta.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)

all_records = []

for setlist in setlists_data:
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    for song in canciones:
        norm_song = normalize(song)
        album = song_to_album.get(norm_song, "Ineditos")
        color = song_to_color.get(norm_song, "#000000")
        all_records.append({
            "fecha": current_date,
            "canción": song.title(),
            "álbum": album,
            "color": color  # Solo cuenta 1 para este concierto
        })

# Super DataFrame con todas las fechas y canciones (sin acumulativo)
df_super = pd.DataFrame(all_records)
df_super




,fecha,canción,álbum,color
0,2025-05-25,Unbekannt,Civilizacion,#d7c391
1,2025-05-25,Desde Lejos No Se Ve,Azul,#0047a5
2,2025-05-25,Ruleta,Verde Paisaje del Infierno,#648e5c
3,2025-05-25,Difícil,Civilizacion,#d7c391
4,2025-05-25,Media Caña,Verde Paisaje del Infierno,#648e5c
...,...,...,...,...
416,2024-12-14,Muévelo,3er Arco,#ead936
417,2024-12-14,El Farolito,3er Arco,#ead936
418,2024-12-14,Finale,Azul,#0047a5
419,2024-12-14,Cruel,Chac Tu Chac,#1e2830


In [167]:
# 1. Normaliza los nombres de las canciones en ambos DataFrames para hacer el merge
df_super['cancion_norm'] = df_super['canción'].apply(normalize)
df_popularidad['cancion_norm'] = df_popularidad['canción'].apply(normalize)

# 2. Une la popularidad a cada canción tocada en cada concierto
df_super = df_super.merge(
    df_popularidad[['cancion_norm', 'indice_popularidad']],
    on='cancion_norm',
    how='left'
)


In [168]:
# 3. Calcula el promedio de popularidad por concierto (por fecha)
df_conciertos_pop = df_super.groupby('fecha')['indice_popularidad'].mean().reset_index()
df_conciertos_pop = df_conciertos_pop.sort_values('indice_popularidad', ascending=False)


# 4. Ahora puedes ver los conciertos más "mainstream" y más "alternativos"
print("Recitales ordenados de mayor a menor popularidad en promedio:")
print(df_conciertos_pop.sort_values('indice_popularidad', ascending=False))

Recitales ordenados de mayor a menor popularidad en promedio:
         fecha  indice_popularidad
7   2025-02-16            0.832397
0   2024-12-14            0.790725
10  2025-05-03            0.761230
8   2025-04-13            0.757790
11  2025-05-10            0.751087
9   2025-04-26            0.705820
1   2024-12-15            0.666556
3   2024-12-21            0.653611
5   2025-01-25            0.610675
2   2024-12-18            0.585763
6   2025-01-26            0.564043
4   2024-12-22            0.562661
13  2025-05-25            0.474778
12  2025-05-24            0.332260


In [169]:
df_conciertos_pop

,fecha,indice_popularidad
7,2025-02-16,0.832397
0,2024-12-14,0.790725
10,2025-05-03,0.761230
8,2025-04-13,0.757790
11,2025-05-10,0.751087
9,2025-04-26,0.705820
1,2024-12-15,0.666556
3,2024-12-21,0.653611
5,2025-01-25,0.610675
2,2024-12-18,0.585763


In [170]:
df_conciertos_pop.to_excel('recitales_ordenados_por_popularidad.xlsx',index = False, header=True)